# Conditioning is Relative, Relative Error

Define $\Delta x= fl(x)-x$

Absolute Error: $f(x+\Delta x) - f(x)$

Relative Error: $\frac{\text{Abs. Error}}{f(x)}$

Absolute Roundoff: $\Delta x$

Relative Roundoff: $\frac{\text{Abs. Roundoff}}{x}$

Relative Error divided by Relative Roundoff: 

$$\frac{\frac{f(x+\Delta x)-f(x)}{f(x)}}{\frac{\Delta x}{x}}$$

Let's simulate this condition number calculation. We will use `Float64` as exact arithmetic and simulate floating point numbers with `Float32` 

Imagine that `fl(x) = Float32(x::Float64)`

In [10]:
Float32(1.4) - 1.4

-2.3841857821338408e-8

In [11]:
eps(Float32)

1.1920929f-7

In [12]:
Float32(1.4) - Float32(1.4)

0.0f0

In [13]:
Float16(1.4) - Float32(1.4)

0.00039064884f0

In [14]:
relerr(f, x, Δx) = abs(f(x+Δx) - f(x)) / abs(f(x))
roundoff(T::Type, x) = T(x) - x
relroundoff(T::Type, x) = abs(roundoff(T, x)) / abs(x)

relroundoff (generic function with 1 method)

In [15]:
relerr(x -> x+1, 1, eps())

0.0

In [16]:
relerr(x -> x-1, 2, eps())

0.0

In [17]:
relroundoff(Float32, 1.4)

1.7029898443813148e-8

In [18]:
relroundoff(Float16, 1.4)

0.0002790178571429206

In [19]:
roundoff(Float32, 1.4)

-2.3841857821338408e-8

In [20]:
relerr(x->x+1, 1, roundoff(Float32, 1.0))
# there is no relative error here because 1.0 is exact in all Float types

0.0

In [21]:
relerr(x->x+1, 1/2, roundoff(Float32, 1/2))
# same for any 1/2^d

0.0

In [22]:
relerr(x->x+1, 0.17, roundoff(Float32, 0.17))


1.5283242857401644e-9

In [23]:
relerr(x->x+1, -0.17, roundoff(Float32, -0.17))


2.154384702763482e-9

In [24]:
relerr(x->x+1, -0.17, roundoff(Float32, -0.17))


2.154384702763482e-9

In [62]:
using PrettyTables

In [103]:
pretty_table(
map(xs) do x
    fx = x - 1
    fx̃ = (x + roundoff(Float32, x)) - 1
    err = relerr(x->x-1, x, roundoff(Float32, x))
    cond = err/relroundoff(Float32, x)
    return (x=x, fx=fx, fx̃=fx̃, var"rel. err."=err, κ̂=cond)
    end,
)

┌─────────┬─────────────┬─────────────┬─────────────┬───────────┐
│       x │          fx │          fx̃ │   rel. err. │         κ̂ │
│ Float64 │     Float64 │     Float64 │     Float64 │   Float64 │
├─────────┼─────────────┼─────────────┼─────────────┼───────────┤
│ 1.33333 │    0.333333 │    0.333333 │  1.19209e-7 │       4.0 │
│ 1.11111 │    0.111111 │    0.111111 │  4.76837e-7 │      10.0 │
│ 1.03704 │    0.037037 │    0.037037 │  5.96046e-7 │      28.0 │
│ 1.01235 │   0.0123457 │   0.0123457 │  5.96046e-7 │      82.0 │
│ 1.00412 │  0.00411523 │  0.00411522 │  5.96046e-7 │     244.0 │
│ 1.00137 │  0.00137174 │  0.00137174 │  5.96046e-7 │     730.0 │
│ 1.00046 │ 0.000457247 │ 0.000457287 │  8.63075e-5 │    2188.0 │
│ 1.00015 │ 0.000152416 │ 0.000152469 │ 0.000347018 │    6562.0 │
│ 1.00005 │  5.08053e-5 │  5.07832e-5 │ 0.000435114 │   19684.0 │
│ 1.00002 │  1.69351e-5 │  1.69277e-5 │ 0.000435114 │   59050.0 │
│ 1.00001 │  5.64503e-6 │  5.60284e-6 │   0.0074743 │  177148.0 │
│     1.

In [61]:
xs = [(3^n+1)/3^n for n in 1:20]
println("x\t\t\tfx\t\t\tfx̃\t\t\t rel. error\t\t\t est. cond")
println("="^120)
for x in xs
    fx = x - 1
    fx̃ = (x + roundoff(Float32, x)) - 1
    err= relerr(x->x-1, x, roundoff(Float32, x))
    cond = err/relroundoff(Float32, x)
    println("$x\t $fx\t $fx̃\t $err\t$cond")
end

x			fx			fx̃			 rel. error			 est. cond
1.3333333333333333	 0.33333333333333326	 0.3333333730697632	 1.1920928977282588e-7	4.000000000000001
1.1111111111111112	 0.11111111111111116	 0.11111116409301758	 4.768371577590356e-7	9.999999999999996
1.037037037037037	 0.03703703703703698	 0.037037014961242676	 5.96046446199595e-7	28.000000000000043
1.0123456790123457	 0.012345679012345734	 0.012345671653747559	 5.960464521947957e-7	81.99999999999964
1.0041152263374487	 0.004115226337448652	 0.0041152238845825195	 5.96046470180398e-7	243.99999999999454
1.0013717421124828	 0.00137174211248281	 0.0013717412948608398	 5.960464162235911e-7	730.000000000023
1.0004572473708275	 0.0004572473708275293	 0.0004572868347167969	 8.630752582818317e-5	2188.0000000004234
1.000152415790276	 0.00015241579027591712	 0.00015246868133544922	 0.0003470182415900121	6561.999999998082
1.0000508052634254	 5.080526342537972e-5	 5.078315734863281e-5	 0.00043511390860863586	19683.999999965574
1.0000169350878085	 1.6935087

Looking at table of output helps you debug. You can notice weird things like how our relative error just jumps to 1.0 and stays there. Let's dive in!

In [76]:
println("x\t\t\trel. error")
println("="^60)
pretty_table(map(xs) do x
    err = relerr(x->x-1, x, roundoff(Float32, x))
    return (x=x, var"rel. error"=err)
    end
)

x			rel. error
┌─────────┬─────────────┐
│       x │  rel. error │
│ Float64 │     Float64 │
├─────────┼─────────────┤
│ 1.33333 │  1.19209e-7 │
│ 1.11111 │  4.76837e-7 │
│ 1.03704 │  5.96046e-7 │
│ 1.01235 │  5.96046e-7 │
│ 1.00412 │  5.96046e-7 │
│ 1.00137 │  5.96046e-7 │
│ 1.00046 │  8.63075e-5 │
│ 1.00015 │ 0.000347018 │
│ 1.00005 │ 0.000435114 │
│ 1.00002 │ 0.000435114 │
│ 1.00001 │   0.0074743 │
│     1.0 │   0.0136433 │
│     1.0 │   0.0497094 │
│     1.0 │    0.140349 │
│     1.0 │    0.710523 │
│     1.0 │         1.0 │
│     1.0 │         1.0 │
│     1.0 │         1.0 │
│     1.0 │         1.0 │
│     1.0 │         1.0 │
└─────────┴─────────────┘


Our relative error is growing smoothing until it suddenly jumps to 1. What happened?

When $\Delta$ drops below `eps(Float32)/2`, our $\tilde{x}$ starts rounding to 1 in 32 bits.

In [29]:
eps(Float32)/2

5.9604645f-8

In [51]:
println("x\t\t\t Δx\t\t\t Float32(x)")
print("="^70)
for xᵢ in xs
    print("\n$xᵢ\t $(xᵢ-1)\t $(Float32(xᵢ))")
    if xᵢ - 1 < eps(Float32)/2
        print("\t\t #", "  Δx < ϵₘ/2 !  ")
    end
end

x			 Δx			 Float32(x)
1.3333333333333333	 0.33333333333333326	 1.3333334
1.1111111111111112	 0.11111111111111116	 1.1111112
1.037037037037037	 0.03703703703703698	 1.037037
1.0123456790123457	 0.012345679012345734	 1.0123457
1.0041152263374487	 0.004115226337448652	 1.0041152
1.0013717421124828	 0.00137174211248281	 1.0013717
1.0004572473708275	 0.0004572473708275293	 1.0004573
1.000152415790276	 0.00015241579027591712	 1.0001525
1.0000508052634254	 5.080526342537972e-5	 1.0000508
1.0000169350878085	 1.6935087808533922e-5	 1.0000169
1.0000056450292696	 5.645029269585322e-6	 1.0000056
1.0000018816764231	 1.8816764231210925e-6	 1.0000019
1.0000006272254744	 6.272254744477124e-7	 1.0000006
1.000000209075158	 2.090751580752226e-7	 1.0000002
1.0000000696917193	 6.969171928439266e-8	 1.0000001
1.000000023230573	 2.3230573020782685e-8	 1.0		 #  Δx < ϵₘ/2 !  
1.0000000077435243	 7.743524266246027e-9	 1.0		 #  Δx < ϵₘ/2 !  
1.0000000025811748	 2.5811748294302106e-9	 1.0		 #  Δx < ϵₘ/2 !  
1.000

In [98]:
pretty_table(map(xs) do x return (x=x, Δx = x-1, var"Float32(x)"=Float32(x)) end, 
    highlighters= Highlighter((row,i,j) -> row[i].Δx < eps(Float32)/2, crayon"bold red"))

┌─────────┬─────────────┬────────────┐
│       x │          Δx │ Float32(x) │
│ Float64 │     Float64 │    Float32 │
├─────────┼─────────────┼────────────┤
│ 1.33333 │    0.333333 │    1.33333 │
│ 1.11111 │    0.111111 │    1.11111 │
│ 1.03704 │    0.037037 │    1.03704 │
│ 1.01235 │   0.0123457 │    1.01235 │
│ 1.00412 │  0.00411523 │    1.00412 │
│ 1.00137 │  0.00137174 │    1.00137 │
│ 1.00046 │ 0.000457247 │    1.00046 │
│ 1.00015 │ 0.000152416 │    1.00015 │
│ 1.00005 │  5.08053e-5 │    1.00005 │
│ 1.00002 │  1.69351e-5 │    1.00002 │
│ 1.00001 │  5.64503e-6 │    1.00001 │
│     1.0 │  1.88168e-6 │        1.0 │
│     1.0 │  6.27225e-7 │        1.0 │
│     1.0 │  2.09075e-7 │        1.0 │
│     1.0 │  6.96917e-8 │        1.0 │
│     1.0 │  2.32306e-8 │        1.0 │
│     1.0 │  7.74352e-9 │        1.0 │
│     1.0 │  2.58117e-9 │        1.0 │
│     1.0 │ 8.60392e-10 │        1.0 │
│     1.0 │ 2.86797e-10 │        1.0 │
└─────────┴─────────────┴────────────┘


In [54]:
println("x\t\t\tfx\t\t\tfx̃\t\t\t rel. error")
println("="^100)
for xᵢ in xs
    x  = xᵢ
    fx = 2xᵢ
    fx̃ = 2(xᵢ + roundoff(Float32, xᵢ))
    err= relerr(x->2x, xᵢ, roundoff(Float32, xᵢ))
    println("$x\t $fx\t $fx̃\t $err")
end

x			fx			fx̃			 rel. error
1.3333333333333333	 2.6666666666666665	 2.6666667461395264	 2.9802322443206464e-8
1.1111111111111112	 2.2222222222222223	 2.222222328186035	 4.768371577590358e-8
1.037037037037037	 2.074074074074074	 2.0740740299224854	 2.128737307855693e-8
1.0123456790123457	 2.0246913580246915	 2.024691343307495	 7.268859173107296e-9
1.0041152263374487	 2.0082304526748973	 2.008230447769165	 2.442813402378735e-9
1.0013717421124828	 2.0027434842249656	 2.0027434825897217	 8.165019400322909e-10
1.0004572473708275	 2.000914494741655	 2.0009145736694336	 3.944585275510351e-8
1.000152415790276	 2.000304831580552	 2.000304937362671	 5.2882999327966096e-8
1.0000508052634254	 2.0001016105268508	 2.0001015663146973	 2.2104953698912663e-8
1.0000169350878085	 2.000033870175617	 2.0000338554382324	 7.368567535646077e-9
1.0000056450292696	 2.000011290058539	 2.0000112056732178	 4.21924225211433e-8
1.0000018816764231	 2.0000037633528462	 2.0000038146972656	 2.567216138470668e-8
1.0000006

In [106]:
pretty_table(
map(xs) do x
    fx = 2x
    fx̃ = 2(x + roundoff(Float32, x))
    err = relerr(x->2x, x, roundoff(Float32, x))
    cond = err/relroundoff(Float32, x)
    return (x=x, roundoff=roundoff(Float32, x), fx=fx, fx̃=fx̃, var"rel. err."=err, κ̂=cond)
    end,
)

┌─────────┬──────────────┬─────────┬─────────┬─────────────┬─────────┐
│       x │     roundoff │      fx │      fx̃ │   rel. err. │       κ̂ │
│ Float64 │      Float64 │ Float64 │ Float64 │     Float64 │ Float64 │
├─────────┼──────────────┼─────────┼─────────┼─────────────┼─────────┤
│ 1.33333 │   3.97364e-8 │ 2.66667 │ 2.66667 │  2.98023e-8 │     1.0 │
│ 1.11111 │   5.29819e-8 │ 2.22222 │ 2.22222 │  4.76837e-8 │     1.0 │
│ 1.03704 │  -2.20758e-8 │ 2.07407 │ 2.07407 │  2.12874e-8 │     1.0 │
│ 1.01235 │   -7.3586e-9 │ 2.02469 │ 2.02469 │  7.26886e-9 │     1.0 │
│ 1.00412 │  -2.45287e-9 │ 2.00823 │ 2.00823 │  2.44281e-9 │     1.0 │
│ 1.00137 │ -8.17622e-10 │ 2.00274 │ 2.00274 │ 8.16502e-10 │     1.0 │
│ 1.00046 │   3.94639e-8 │ 2.00091 │ 2.00091 │  3.94459e-8 │     1.0 │
│ 1.00015 │   5.28911e-8 │  2.0003 │  2.0003 │   5.2883e-8 │     1.0 │
│ 1.00005 │  -2.21061e-8 │  2.0001 │  2.0001 │   2.2105e-8 │     1.0 │
│ 1.00002 │  -7.36869e-9 │ 2.00003 │ 2.00003 │  7.36857e-9 │     1.0 │
│ 1.